In [11]:
import pandas as pd

# Set display options to show all columns (and optionally all rows)
pd.set_option('display.max_columns', None)  # None means all columns will be shown
#pd.set_option('display.max_rows', None)     # None means all rows will be shown, use with caution

In [ ]:
# pip install pandas openpyxl

In [30]:
file_path = r'data\Book3.xlsx'

In [19]:
column = ['Name', 'PIS ID','AutoCAD', 'AutoCAD_MEP','AutoCAD Plant 3D',	'AutoCAD Civil 3D',	'AutoCAD Raster Design','Revit MEP','RevitACS','Fabrication CADmep','Navisworks','Staad Pro',	'Hap',	'Microstation',	'Rebar CAD',	'Tekla Structure',	'BlueBeam',	'COBie Data',	'BIM 360',	'SysQue',	'MS - Office',	'Architecture',	'Structure','Rebar Technical','Infrastructure','Stuctural Design','Mechanical','Plumbing','Process Piping','Electrical','Mechanical Design','Plumbing Design','Electrical Design','FireProtection']

In [31]:
#df = pd.read_excel(file_path, usecols=column,skiprows=2)
df = pd.read_excel(file_path)

In [43]:
df.columns[:50]

Index(['name', 'pisid', 'autocad', 'autocad_mep', 'autocadplant3d',
       'autocadcivil3d', 'autocadrasterdesign', 'revitmep', 'revitacs',
       'fabricationcadmep', 'navisworks', 'staadpro', 'hap', 'microstation',
       'rebarcad', 'teklastructure', 'bluebeam', 'cobiedata', 'bim360',
       'sysque', 'ms-office', 'architecture', 'structure', 'rebartechnical',
       'infrastructure', 'stucturaldesign', 'mechanical', 'plumbing',
       'processpiping', 'electrical', 'mechanicaldesign', 'plumbingdesign',
       'electricaldesign', 'fireprotection'],
      dtype='object')

In [44]:
software_col = ['name', 'pisid', 'autocad', 'autocad_mep', 'autocadplant3d',
       'autocadcivil3d', 'autocadrasterdesign', 'revitmep', 'revitacs',
       'fabricationcadmep', 'navisworks', 'staadpro', 'hap', 'microstation',
       'rebarcad', 'teklastructure', 'bluebeam', 'cobiedata', 'bim360',
       'sysque', 'ms-office']

In [45]:
trade_col = ['architecture', 'structure', 'rebartechnical',
       'infrastructure', 'stucturaldesign', 'mechanical', 'plumbing',
       'processpiping', 'electrical', 'mechanicaldesign', 'plumbingdesign',
       'electricaldesign', 'fireprotection']

In [46]:
df.head()

,name,pisid,autocad,autocad_mep,autocadplant3d,autocadcivil3d,autocadrasterdesign,revitmep,revitacs,fabricationcadmep,navisworks,staadpro,hap,microstation,rebarcad,teklastructure,bluebeam,cobiedata,bim360,sysque,ms-office,architecture,structure,rebartechnical,infrastructure,stucturaldesign,mechanical,plumbing,processpiping,electrical,mechanicaldesign,plumbingdesign,electricaldesign,fireprotection
0,Samar Sen,PIS00006,3.0,NaN,NaN,NaN,3.0,3.0,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1,NaN,NaN,NaN,NaN,2.0
1,Sanat Kumar Roy,PIS00015,3.0,NaN,NaN,NaN,1.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,2.0,NaN,3.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sourav Dutta,PIS00017,4.0,NaN,NaN,NaN,NaN,2.0,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Ajoy Maji,PIS00023,3.0,NaN,NaN,NaN,NaN,2.0,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0
4,Mantu Naskar,PIS00027,3.0,2.0,NaN,NaN,NaN,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
data = df.copy()

In [57]:
# Function to melt and create ratings for given categories
def melt_ratings(df, id_vars, value_vars, value_name, var_name):
    melted = df.melt(id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)
    # Remove rows with NaN ratings
    melted = melted.dropna(subset=[value_name])
    return melted

In [58]:
# Melt the dataframe for software ratings
software_df = melt_ratings(data, ["name", "pisid"], software_col, "Software Rating", "Software")

# Melt the dataframe for trade ratings
# If there are no trade columns, you can comment out or remove the following lines
trade_df = melt_ratings(data, ["name", "pisid"], trade_col, "Trade Rating", "Trade")

# If you need to combine software and trade ratings, merge the two dataframes
# If you only have software ratings, use `software_df` directly
combined_df = pd.merge(software_df, trade_df, on=["name", "pisid"], how="outer")

# Reset the index of the final dataframe
combined_df.reset_index(drop=True, inplace=True)

In [59]:
combined_df.head()

,name,pisid,Software,Software Rating,Trade,Trade Rating
0,Samar Sen,PIS00006,autocad,3.0,mechanical,3.0
1,Samar Sen,PIS00006,autocad,3.0,plumbing,3.0
2,Samar Sen,PIS00006,autocad,3.0,processpiping,1
3,Samar Sen,PIS00006,autocad,3.0,fireprotection,2.0
4,Samar Sen,PIS00006,autocadrasterdesign,3.0,mechanical,3.0


In [61]:
# combined_df.to_csv('data/PDH_input_details.csv')

In [60]:
combined_df[combined_df["pisid"] == 'PIS00006']

,name,pisid,Software,Software Rating,Trade,Trade Rating
0,Samar Sen,PIS00006,autocad,3.0,mechanical,3.0
1,Samar Sen,PIS00006,autocad,3.0,plumbing,3.0
2,Samar Sen,PIS00006,autocad,3.0,processpiping,1
3,Samar Sen,PIS00006,autocad,3.0,fireprotection,2.0
4,Samar Sen,PIS00006,autocadrasterdesign,3.0,mechanical,3.0
5,Samar Sen,PIS00006,autocadrasterdesign,3.0,plumbing,3.0
6,Samar Sen,PIS00006,autocadrasterdesign,3.0,processpiping,1
7,Samar Sen,PIS00006,autocadrasterdesign,3.0,fireprotection,2.0
8,Samar Sen,PIS00006,revitmep,3.0,mechanical,3.0
9,Samar Sen,PIS00006,revitmep,3.0,plumbing,3.0


In [35]:
# Melt the dataframe to convert software types from columns to rows
melted_dataframe = data.melt(id_vars=["name", "pisid"], 
                                       var_name="SoftwareType", 
                                       value_name="Rating")

In [36]:
# Remove rows with NaN ratings
cleaned_dataframe = melted_dataframe.dropna(subset=['Rating'])

In [40]:
cleaned_dataframe[cleaned_dataframe["pisid"] == 'PIS00006']

,name,pisid,SoftwareType,Rating
0,Samar Sen,PIS00006,autocad,3.0
9484,Samar Sen,PIS00006,autocadrasterdesign,3.0
11855,Samar Sen,PIS00006,revitmep,3.0
16597,Samar Sen,PIS00006,fabricationcadmep,3.0
18968,Samar Sen,PIS00006,navisworks,3.0
33194,Samar Sen,PIS00006,bluebeam,1.0
37936,Samar Sen,PIS00006,bim360,1.0
42678,Samar Sen,PIS00006,ms-office,3.0
56904,Samar Sen,PIS00006,mechanical,3.0
59275,Samar Sen,PIS00006,plumbing,3.0


In [41]:
# Reset the index of the final dataframe
cleaned_dataframe = cleaned_dataframe.reset_index(drop=True)

In [42]:
cleaned_dataframe.head()

,name,pisid,SoftwareType,Rating
0,Samar Sen,PIS00006,autocad,3.0
1,Sanat Kumar Roy,PIS00015,autocad,3.0
2,Sourav Dutta,PIS00017,autocad,4.0
3,Ajoy Maji,PIS00023,autocad,3.0
4,Mantu Naskar,PIS00027,autocad,3.0
